In [2]:
#reference : https://www.zenrows.com/blog/dynamic-web-pages-scraping-python#selecting-elements
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import io


In [2]:
#load trainers names

trainers_names = pd.read_csv('data/trainers_premiership.csv')

In [6]:
'-'.join(trainers_names.Trainer[0].split('_'))

'Mark-Walker-&-Sam-Bergerson'

In [19]:

driver = webdriver.Chrome(ChromeDriverManager().install())
#url = 'https://www.racing.com/trainers/tony-pike-nz/form'
url = 'https://www.racing.com/trainers/andrew-carston-nz/form'
driver.get(url)


In [20]:


driver.title

'404 Not Found - RACING.COM'

In [13]:
from selenium.webdriver.common.by import By
aux = driver.find_elements(By.TAG_NAME, 'title')

In [16]:
aux.sort

<function list.sort(*, key=None, reverse=False)>

In [5]:

from selenium.webdriver.common.by import By
elements = driver.find_elements(By.TAG_NAME, 'table') 
trainers_form_races = [trainers_elements.text for trainers_elements in elements]
#print(trainers_names)
trainers_form_races


["POS\nHORSE/TRAINER/JOCKEY\nTRACK/DATE\nDIST/COND\nPRIZE/CLASS\nWGT\n800M/400M\nMARGIN\nRATING\nODDS\nCOM\n7-18\n14. Cheval D'or (NZ) (17)\nT: \nA.W.Pike\n  J: \nM.J.McNab\nE FM R8\n03 JUN 23\n2200M\nGood 4\n$700K\nG1 QLD OAKS\n56.5kg\n6th / 4th\n6.49L\n84\n$15\nSP\n1-6\nBabylon Berlin (6)\nT: \nA.W.Pike\n  J: \n - \nTE R R3\n04 MAR 23\n1200M\nGood 4\n$93.1K\nG3 KINGS PLATE\n54kg\n-\n-\n-\n$1.1\nSP\n2-12\nBabylon Berlin (9)\nT: \nA.W.Pike\n  J: \n - \nTE R R8\n11 FEB 23\n1400M\nGood 4\n$279.3K\nG1 BCD GRP SPRINT\n57kg\n-\n4.5L\n-\n$8.5\nSP\n1-11\nNot Guilty (NZ)\nT: \nA.W.Pike\n  J: \nM.J.McNab\nTE R R3\n11 FEB 23\n1200M\nGood\n$46.5K\nRATING 75\n59kg\n-\n-\n-\n$3.5\nSP\n2-11\nRuach (NZ)\nT: \nA.W.Pike\n  J: \nJ.K.Riddell\nTE R R2\n18 JAN 23\n1500M\nGood\n$13K\nRATING 75\n58kg\n-\n0.2L\n-\n$7\nSP\n2-11\nBabylon Berlin (8)\nT: \nA.W.Pike\n  J: \n - \nTREN R8\n14 JAN 23\n1200M\nGood 4\n$325.8K\nG1 TELEGRAPH\n56.5kg\n-\n0.4L\n-\n$3.7\nSP\n2-10\nBabylon Berlin (2)\nT: \nA.W.Pike\n  J: \n 

we have the names of the trainers, now we can loop trough them, access the page of each and get their info, but before that, let's define the table making roadmap:

- Split the Data: First, split the string into individual entries based on a unique pattern. In this case, it seems each entry starts with a position like "7-18".

- Parse Each Entry: For each entry, further split the information into its respective columns. This might involve identifying specific patterns or separators in the text.

- Create a Data Structure: Store the parsed data in a structured format like a list of dictionaries or a list of lists, which can then be easily converted into a DataFrame.

- Convert to DataFrame: Use pandas to convert this structured data into a DataFrame for easier manipulation and analysis.

In [8]:
trainers_form_races[0].find('-')

96

Ready for a random knowledge? that number into parentheses after the horse's name is represents the barrier draw or gate for that horse. 

In [29]:
import pandas as pd
import re

# Example function to parse each entry
def parse_entry(entry):
    lines = entry.split('\n')
    return {
        "Position": lines[0],
        "Horse": lines[1],
        "Trainer": lines[3].strip('T: '),
        "Jockey": lines[5].strip('J: '),
        "Track": lines[6],
        "Date": lines[7],
        "Distance": lines[8],
        "Condition": lines[9],
        "Prize":lines[10],
        "Class":lines[11],
        "WGT":lines[12],
        "Pos_800_400":lines[13],
        "Margin":lines[14],
        "Rating":lines[15],
        "Odds":lines[16]
    }

# Split the data into individual entries
# Assuming each entry starts with a pattern like '7-18' followed by a horse name
entries = re.split(r'\n(?=\d+-\d+\n)', trainers_form_races[0])[1:]  # [1:] to skip the header

# Parse each entry
parsed_data = [parse_entry(entry) for entry in entries]

# Create DataFrame
df = pd.DataFrame(parsed_data)

print(df)


   Position                       Horse   Trainer              Jockey  \
0      7-18   14. Cheval D'or (NZ) (17)  A.W.Pike           M.J.McNab   
1       1-6          Babylon Berlin (6)  A.W.Pike                   -   
2      2-12          Babylon Berlin (9)  A.W.Pike                   -   
3      1-11             Not Guilty (NZ)  A.W.Pike           M.J.McNab   
4      2-11                  Ruach (NZ)  A.W.Pike          .K.Riddell   
5      2-11          Babylon Berlin (8)  A.W.Pike                   -   
6      2-10          Babylon Berlin (2)  A.W.Pike                   -   
7      2-10          Babylon Berlin (8)  A.W.Pike    K.Hannan (anull)   
8       1-7                  Letzbeglam  A.W.Pike                   -   
9     13-14       2. Catalyst (NZ) (13)  A.W.Pike            D.M.Lane   
10     1-12          Surely Sacred (NZ)  A.W.Pike  M.Chadwick (anull)   
11    11-14          10. Letzbeglam (2)  A.W.Pike            K.McEvoy   
12     9-11       9. Babylon Berlin (5)  A.W.Pike  

In [21]:
df.to_csv('previous_races.csv',index=False)